# ChatGPT不会算算术（现在会算数了）

In [1]:
import os
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=2048, temperature=0.5)
multiply_prompt = PromptTemplate(template="请计算一下{question}是多少?", input_variables=["question"])
math_chain = LLMChain(llm=llm, prompt=multiply_prompt, output_key="answer")
answer = math_chain.run(question="352乘以493")

print("OpenAI API 说答案是:", answer)

python_answer = 352 * 493
print("Python 说答案是:", python_answer)


multiply_by_python_prompt = PromptTemplate(template="请写一段Python代码，计算{question}?只输出原生代码", input_variables=["question"])
python_chain = LLMChain(llm=llm, prompt=multiply_by_python_prompt, output_key="answer")
answer = python_chain.run(question="352乘以493")
print(answer)

from langchain.utilities import PythonREPL
python_repl = PythonREPL()
result = python_repl.run(answer)
print(result)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


OpenAI API 说答案是: 352乘以493等于173536。
Python 说答案是: 173536


Python REPL can execute arbitrary code. Use with caution.


result = 352 * 493
print(result)
173536



# 通过LLMMathChain算算术

In [2]:
!pip install numexpr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 312.8 kB/s eta 0:00:0000:0100:01


In [3]:
from langchain import LLMMathChain

llm_math = LLMMathChain(llm=llm, verbose=True)
result = llm_math.run("请计算一下352乘以493是多少?")
print(result)

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new LLMMathChain chain...
请计算一下352乘以493是多少?```text
352 * 493
```
...numexpr.evaluate("352 * 493")...

Answer: 173536
> Finished chain.
Answer: 173536


# 通过Google搜索查天气

In [4]:
from langchain.chains import LLMRequestsChain

template = """在 >>> 和 <<< 直接是来自Google的原始搜索结果.
请把对于问题 '{query}' 的答案从里面提取出来，使用json格式输出(weather、temperature.min、temperature.max、wind.direction、wind.level)。如果里面没有相关信息的话就说“找不到”
请使用如下格式：
Extracted:<answer or "找不到">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    template=template,
    input_variables=["query", "requests_result"],
)

requests_chain = LLMRequestsChain(llm_chain=LLMChain(llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0), prompt=PROMPT))
question="今天上海的天气怎么样？"

inputs = {
    "query":question,
    "url": "https://www.baidu.com/s?wd=" + question.replace(" ", "+")
    # "url": "https://www.google.com/search?q=" + question.replace(" ", "+")
}

result=requests_chain(inputs)
print(result)
print(result['output'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': '今天上海的天气怎么样？', 'url': 'https://www.baidu.com/s?wd=今天上海的天气怎么样？', 'output': '{\n    "weather": "多云",\n    "temperature": {\n        "min": 19,\n        "max": 27\n    },\n    "wind": {\n        "direction": "东南风",\n        "level": "2级"\n    }\n}'}
{
    "weather": "多云",
    "temperature": {
        "min": 19,
        "max": 27
    },
    "wind": {
        "direction": "东南风",
        "level": "2级"
    }
}


# 通过TransformChain运行自定义函数

In [5]:
from langchain.chains import TransformChain, SequentialChain

def transform_func(inputs: dict) -> dict:
    text = inputs["output"]
    if (text == "找不到") :
        text = {}
    return {"weather_info" : text}


transformation_chain = TransformChain(
    input_variables=["output"],
    output_variables=["weather_info"],
    transform=transform_func
)

final_chain = SequentialChain(
    chains=[requests_chain, transformation_chain],
    input_variables=["query", "url"],
    output_variables=["weather_info"]
)

final_result = final_chain.run(inputs)
print(final_result)

{
    "weather": "多云",
    "temperature": {
        "min": 19,
        "max": 27
    },
    "wind": {
        "direction": "东南风",
        "level": "2级"
    }
}


# 通过VectorDBQA做问答

In [6]:
!pip install spacy
!pip install faiss-cpu
!python3 -m spacy download zh_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 2.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 MB 1.5 MB/s eta 0:00:0000:0100:01m
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')


In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import SpacyTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.document_loaders import TextLoader
from langchain_core.vectorstores import VectorStoreRetriever

llm = OpenAI(temperature=0)

loader = TextLoader('./data/ecommerce_faq.txt')
documents = loader.load()

text_splitter = SpacyTextSplitter(chunk_size=1000, pipeline="zh_core_web_sm")
texts = text_splitter.split_documents(documents)


embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_documents(texts, embeddings)

retriever = VectorStoreRetriever(vectorstore=docsearch)

faq_chain = RetrievalQA.from_llm(llm=llm, retriever=retriever)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
question = "请问你们的货，能送到三亚吗？大概需要几天？"
result = faq_chain.run(question)
print(result)

 我们支持海南省的配送，一般情况下，大部分城市的订单在2-3个工作日内送达，偏远地区可能需要5-7个工作日。具体送货时间可能因订单商品、配送地址和物流公司而异。


In [9]:
question = "请问你们的退货政策是怎么样的？"
result = faq_chain.run(question)
print(result)

 自收到商品之日起7天内，如产品未使用、包装完好，您可以申请退货。某些特殊商品可能不支持退货，请在购买前查看商品详情页面的退货政策。


In [10]:
!pip list

Package                                 Version
--------------------------------------- --------------
aiofiles                                22.1.0
aiohttp                                 3.9.5
aiosignal                               1.3.1
aiosqlite                               0.20.0
altair                                  5.3.0
annotated-types                         0.6.0
anyio                                   4.3.0
argon2-cffi                             23.1.0
argon2-cffi-bindings                    21.2.0
arrow                                   1.3.0
astroid                                 3.1.0
asttokens                               2.4.1
async-timeout                           4.0.3
attrs                                   23.2.0
autopep8                                2.0.4
Babel                                   2.14.0
backoff                                 2.2.1
beautifulsoup4                          4.12.3
bleach                                  6.1.0
blis            